# 🛡️ Engineering Trustworthy AI: The Decoupled Shield Pattern

**Red Hat Summit Connect 2025**

In [1]:
# Install and import dependencies
!pip install llama-stack-client pandas ipywidgets -q

from llama_stack_client import LlamaStackClient
from llama_stack_client.lib.agents.agent import Agent
from uuid import uuid4
import logging

# Import our demo helpers
from shield_demo_helpers import (
    ShieldMetrics,
    show_hero_banner,
    show_attack_surface,
    show_result_card,
    show_comparison_matrix,
    show_compliance_savings,
    create_interactive_tester,
    TEST_PROMPTS,
    SHIELD_CONFIG
)

# Suppress verbose logging
logging.getLogger("httpx").setLevel(logging.WARNING)
logging.getLogger("llama_stack_client").setLevel(logging.WARNING)

print("✅ Environment ready")


[notice] A new release of pip is available: 24.2 -> 25.2
[notice] To update, run: pip install --upgrade pip
✅ Environment ready


In [2]:
# Show hero banner
show_hero_banner()

## Configuration

In [3]:
# Configure your deployment
LLAMASTACK_URL = "http://llamastack-trustyai-fms-service.summit-connect-2025.svc.cluster.local:8321"
MODEL_NAME = "tinyllama-1b"

# Connect to LlamaStack
client = LlamaStackClient(base_url=LLAMASTACK_URL)
metrics = ShieldMetrics()

print(f"✅ Connected to {LLAMASTACK_URL}")
print(f"📊 Model: {MODEL_NAME}")

✅ Connected to http://llamastack-trustyai-fms-service.summit-connect-2025.svc.cluster.local:8321
📊 Model: tinyllama-1b


## Part 1: The Trust Gap - Unprotected Agent

In [4]:
show_attack_surface({'name': 'Unprotected Agent', 'input': False, 'output': False})

In [8]:
# Demo the trust gap
critical_test = TEST_PROMPTS["pii_multiple"]
print(f"📨 User: {critical_test['prompt'][:200]}...\n")

response = client.chat.completions.create(
    model=MODEL_NAME,
    messages=[{"role": "user", "content": critical_test['prompt']}],
    stream=False
)

show_result_card(
    "PROCESSED WITHOUT VALIDATION",
    "error",
    response.choices[0].message.content[:500] + "...",
    "⚠️ PII was processed without any checks!"
)

📨 User: Hi, I'm having issues. My email is jane@company.com, SSN is 987-65-4321, and card 5105-1051-0510-5100....



## Part 2: Register TrustyAI Shields

In [9]:
# Register the shield
try:
    client.shields.register(**SHIELD_CONFIG)
    show_result_card("Shield Registered", "allowed", "TrustyAI PII shield is active")
except Exception as e:
    if "already exists" not in str(e).lower():
        raise

## Part 3: Input Shields

In [10]:
show_attack_surface({'name': 'Input Shield Only', 'input': True, 'output': False})

agent_input = Agent(
    client,
    model=MODEL_NAME,
    instructions='You are a helpful customer service agent.',
    input_shields=['pii_shield'],
    output_shields=[],
    enable_session_persistence=False
)

print("🛡️ Agent created with input shield")

🛡️ Agent created with input shield


In [11]:
# Test input shield
session = agent_input.create_session(f"session-{uuid4()}")
response = agent_input.create_turn(
    messages=[{"role": "user", "content": TEST_PROMPTS["pii_multiple"]["prompt"]}],
    session_id=session,
    stream=False
)

# Show result
for step in response.steps:
    if step.step_type == 'shield_call' and step.violation:
        show_result_card("BLOCKED BY INPUT SHIELD", "blocked", step.violation.user_message)
        metrics.record(blocked=True)
        break

## Part 4: Output Shields

In [12]:
show_attack_surface({'name': 'Output Shield Only', 'input': False, 'output': True})

agent_output = Agent(
    client,
    model=MODEL_NAME,
    instructions='You are a helpful customer service agent.',
    input_shields=[],
    output_shields=['pii_shield'],
    enable_session_persistence=False
)

print("🛡️ Agent created with output shield")

🛡️ Agent created with output shield


## Part 5: Defense-in-Depth - Both Shields

In [13]:
show_attack_surface({'name': 'Both Shields (Recommended)', 'input': True, 'output': True})

agent_both = Agent(
    client,
    model=MODEL_NAME,
    instructions='You are a helpful customer service agent.',
    input_shields=['pii_shield'],
    output_shields=['pii_shield'],
    enable_session_persistence=False
)

print("🛡️🛡️ Agent created with maximum protection")

🛡️🛡️ Agent created with maximum protection


## Part 6: The Complete Picture

In [14]:
show_comparison_matrix()

Attack Scenario,No Shields,Input Only,Output Only,Both (Recommended)
Email PIIjohn@example.com,❌ EXPOSED,🛡️ Blocked,⚠️ Partial,🛡️ Blocked
SSN123-45-6789,❌ EXPOSED,🛡️ Blocked,⚠️ Partial,🛡️ Blocked
Credit Card4532-1234-5678-9010,❌ EXPOSED,🛡️ Blocked,⚠️ Partial,🛡️ Blocked
Multiple PIIemail + SSN + card,❌ EXPOSED,🛡️ Blocked,⚠️ Partial,🛡️ Blocked
Safe QueryHow do I reset password?,✅ Allowed,✅ Allowed,✅ Allowed,✅ Allowed


## Part 7: Real-World Impact

In [15]:
show_compliance_savings()

## Part 8: Try It Yourself

In [16]:
create_interactive_tester(client)

Textarea(value='', layout=Layout(height='80px', width='100%'), placeholder='Enter a test message (try includin…

Button(button_style='primary', description='🛡️ Test with Shield', layout=Layout(width='200px'), style=ButtonSt…

Output()

In [17]:
# Final metrics
metrics.display()